In [1]:
from pathlib import Path
import pandas as pd
import json
from datasets import load_dataset

/home/user/kew/anaconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
xquad_task_description_en = """
### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.

Passage:

{context}

Question:

{question}

Note: Your answer should be directly extracted from the passage and be a single entity, name, or number. Provide the answer in quotations.### Assistant: Based on the passage, the answer to the question is "
"""

xquad_task_description_de = """
### Human: Beantworte die Frage anhand des vorgegebenen Textes. Deine Antwort muss direkt aus dem Text entnommen sein und aus einer einzelnen Einheit, einem Namen oder einer Zahl bestehen. Deine Antwort darf nicht aus einem Satz bestehen.

Text:

{context}

Frage:

{question}

Hinweis: Deine Antwort muss direkt aus dem Text entnommen werden und aus einer einzigen Einheit, einem Namen oder einer Zahl bestehen. Setze die Antwort in Anführungs- und Schlusszeichen.### Assistant: Anhand des Textes lautet die Antwort auf die Frage „
"""

xquad_task_description_es = """
### Human: Responda a la pregunta a partir del pasaje dado. Tu respuesta debe ser extraída directamente del texto, y debe ser una sola entidad, nombre o número, no una frase.

Texto:

{context}

Pregunta:

{question}

Nota: Tu respuesta debe ser extraída directamente del texto y debe ser una sola entidad, nombre o número. Presenta la respuesta entre comillas.### Assistant: Basándose en el texto, la respuesta a la pregunta es «
"""

xquad_task_description_ru = """
### Human: Ответь на вопрос из приведенного отрывка. Твой ответ должен быть непосредственно извлечен из отрывка и содержать только один элемент, будь это предмет, имя или число. Ответ не должен быть целым предложением.

Отрывок:

{context}

Вопрос:

{question}

Внимание: Ваш ответ должен быть непосредственно взят из отрывка и представлять собой один предмет, имя или число. Укажите ответ в кавычках.### Assistant: Исходя из отрывка, ответ на вопрос: «
"""

xquad_task_description_el = """
### Human: απάντησε στην ερώτηση από το δοθέν απόσπασμα. Η απάντησή σου πρέπει να ληφθεί επί λέξει από το απόσπασμα, και πρέπει να είναι μια οντότητα, ένας αριθμός ή ένα όνομα, όχι μια πρόταση.

Απόσπασμα:

{context}

Ερώτηση:

{question}

Σημείωση: Η απάντησή σου θα πρέπει να ληφθεί επί λέξει από το απόσπασμα και πρέπει να είναι μια οντότητα, ένα όνομα ή ένας αριθμός. Δώσε την απάντηση σε εισαγωγικά.### Assistant: Δεδομένου του κειμένου, η απάντηση στην ερώτηση είναι «
"""

xquad_task_description_zh = """
### Human：请根据给定的段落回答问题。你的答案应直接提取自该段落，且应该为一个专有名词、名字或数字，而不是一个句子。

段落：

{context}

问题：

{question}

请注意：你的答案应直接摘自该段落， 且为一个专有名词，名字或数字。请在引号中给出答案。 ### Assistant：根据这段文字，该问题的答案是 "
"""

xquad_task_description_vi = """
### Human: Trả lời câu hỏi trong đoạn văn cho sẵn. Câu trả lời của bạn phải được trích trực tiếp từ đoạn văn và phải là một thực thể, tên hoặc số duy nhất chứ không phải một câu.

Đoạn văn:

{context}

Câu hỏi:

{question}

Lưu ý: Câu trả lời của bạn phải được trích trực tiếp từ đoạn văn và phải là một thực thể, tên hoặc số duy nhất. Cung cấp câu trả lời trong dấu ngoặc kép.### Assistant: Dựa trên đoạn văn, câu trả lời cho câu hỏi là 
"""

xquad_task_description_ar = """
### Human:

أجب السؤال التالي من القطعة. يجب أن تستخرج إجابتك مباشرة من القطعة، ويجب أن تكون كلمة واحدة (اسمًا أو رقمًا) وليس جملة. 

:القطعة

{context}

:سؤال

{question}

ملاحظة: يجب أن تكون إجابتك مستخرجة مباشرة من القطعة وأن تكون  كلمة واحدة (اسمًا أو رقمًا) . أكتب الإجابة في شكل اقتباس

### Assistant:

” بناء على القطعة فإن إجابة السؤال هي 
"""

xquad_task_description_th = """
### Human: ตอบคำถามจากข้อความที่ให้ไว้ คำตอบของคุณควรดึงมาจากข้อความโดยตรง และควรเป็นข้อมูล ชื่อ หรือตัวเลขเพียงรายการเดียว ไม่ใช่ประโยค

ทางเดิน:

{context}

คำถาม:

{question}

หมายเหตุ: คำตอบของคุณควรดึงมาจากข้อความนี้โดยตรงและเป็นเอนทิตี ชื่อ หรือหมายเลขเดียว ให้คำตอบเป็นเครื่องหมายคำพูด### Assistant: จากเนื้อเรื่อง คำตอบของคำถามคือ "
"""

xquad_task_description_tr = """
### Human: Soruyu verilen metne göre cevaplayın. Cevabınız doğrudan metinden alınmalı ve bir cümle değil, tek bir varlık, isim veya sayı olmalıdır.

Metin:

{context}

Soru:

{question}

Not: Cevabınız doğrudan metinden alınmış olmalı ve tek bir varlık, isim veya sayı olmalıdır. Yanıtı tırnak işareti içinde belirtin.### Assistant: Parçaya göre sorunun yanıtı "
"""

xquad_task_description_hi = """
### Human: दिए गए गद्यांश से प्रश्न का उत्तर दीजिए। आपका उत्तर सीधे अनुच्छेद से निकाला जाना चाहिए, और यह एक इकाई, नाम या संख्या होना चाहिए, वाक्य नहीं।

रास्ता:

{context}

सवाल:

{question}

ध्यान दें: आपका उत्तर सीधे गद्यांश से निकाला जाना चाहिए और एक इकाई, नाम या संख्या होना चाहिए। उद्धरणों में उत्तर प्रदान करें। ### Assistant: गद्यांश के आधार पर, प्रश्न का उत्तर है "
"""

language_prompts = {
    'en': xquad_task_description_en,
    'es': xquad_task_description_es,
    'de': xquad_task_description_de,
    'ru': xquad_task_description_ru,
    'hi': xquad_task_description_hi,
    'el': xquad_task_description_el,
    'zh': xquad_task_description_zh,
    'vi': xquad_task_description_vi,
    'ar': xquad_task_description_ar,
    'th': xquad_task_description_th,
    'tr': xquad_task_description_tr,
}




In [5]:
output_dir = Path('../data/xcsqa')

for lang_tag in language_prompts.keys():
    data = load_dataset('xquad', f'xquad.{lang_tag}')['validation']
    print(data)
    with open(output_dir / f'xquad_dev_en_{lang_tag}.json', 'w', encoding='utf8') as en_outf:
        with open(output_dir / f'xquad_dev_{lang_tag}_{lang_tag}.json', 'w', encoding='utf8') as x_outf:
            for i, row in enumerate(data):
                # print(row)
                context = row['context'].strip()
                question = row['question'].strip()
                answer = row['answers']['text'][0].strip()
                if len(row['answers']['text']) > 1:
                    print(f'Warning: {len(row["answers"]["text"])} answers found for {lang_tag} data.')

                en_formatted_item = {
                    'instruction': xquad_task_description_en.format(context=context, question=question).strip(),
                    'answer': answer,
                }

                x_formatted_item = {
                    'instruction': language_prompts[lang_tag].format(context=context, question=question).strip(),
                    'answer': answer,
                }

                if i == 0:
                    print(en_formatted_item)
                    print(x_formatted_item)


                en_outf.write(f'{json.dumps(en_formatted_item, ensure_ascii=False)}\n')
                x_outf.write(f'{json.dumps(x_formatted_item, ensure_ascii=False)}\n')
            
        print(f'Finished writing {lang_tag} data. Wrote {len(data)} items.')

Found cached dataset xquad (/home/user/kew/.cache/huggingface/datasets/xquad/xquad.el/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)
100%|██████████| 1/1 [00:00<00:00, 19.89it/s]


Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})
{'instruction': '### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.\n\nPassage:\n\nΗ άμυνα των Καρολίνα Πάνθερς παρέδωσε μόνο 308 πόντους, και είναι έκτη στην κατάταξη του πρωταθλήματος, ενώ προηγείται στο NFL με 24 interceptions και διαθέτει επίσης τέσσερις επιλογές για Pro Bowl. Ο αμυντικός tackle Kawann Short (Pro Bowl) ήταν πρώτος στην ομάδα του με 11 sacks, ενώ επίσης επέβαλε τρία fumbles και ανάκτησε την κατοχή δύο φορές. Ο άλλος αμυντικός Mario Addison πρόσθεσε 6½ sacks. Η γραμμή των Πάνθερς είχε επίσης τον βετεράνο αμυντικό end Jared Allen, 5 φορές pro bowler, που είναι επίσης ο κορυφαίος ενεργός στο NFL παίκτης σε sack, με σύνολο 136, μαζί με τον αμυντικό end Kony Ealy, που είχε 5 sack σε μόλις 9 start. Πίσω τους, οι δύο από τους τρεις βασικούς linebackers των Πά